In [1]:
import imp
imp.reload(lamb.parsing)
reload_lamb()

# Continuations and scope
## Notebook author: Kyle Rawlins

Based on Chris Barker, "[Continuations and the Nature of Quantification](http://www.semanticsarchive.net/Archive/902ad5f7/barker.continuations.pdf)", Natural Language Semantics, 2002.

This notebook implements a version of Barker's account of scope using continuations.  Barker develops a semantic account of quantifier scope that does not rely at all on LF manipulations etc., but rather complicates the types of lexical items and composition rules in a systematic way to make them more general.  This notebook develops the analysis in roughly the sequence found in Barker's paper; first I show how to continuize ordinary meanings, and how to write a quantifier in this framework.  I then turn to scope ambiguities, scope freezing, and finally inverse linking / QDP-within-QDP cases.

Initial note: because I do not use rule-to-rule translation, and instead use general composition rules, the predictions of the system in terms of possible composition paths are somewhat closer to those developed in Yusuke Kubota and Wataru Uegaki, "[Continuation-based semantics for Conventional Implicatures: The case of Japanese benefactives](http://elanguage.net/journals/salt/article/view/19.18)", Proceedings of SALT 19, 2009.  Basically, a system with general rules tends to overgenerate possible composition paths that converge on the same meaning; as far as I am aware the resultant S-level meanings (filtered to be the appropriate type) are the same as Barker's system.  I will note several places where this is relevant.

Continuations are notoriously hard to explain, and I will do so here mainly by example.  I recommend using the notebook to inspect objects that are mysterious, check on the details of derivations, etc.

An ordinary meaning can be continuized by turning it into a function from its continuation to itself applied as an argument to the continuation.  The continuation represents something like the "next state" of the computation.  The continuation of X is always a function from X to type t; the output type is because all derivations (by assumption) end in ordinary type t.

The effect is perhaps easiest to see when continuizing type t, exemplified by `raining1` (the proposition that it is raining) and `raining2` (its continuized form) below.  The continuation of something of ordinary type `t` is a function that maps that type to the output type, namely `t`.  Crucially, if the identity function for `<t,t>` is combined via function application with a continuized denotation of this type, we always get back its ordinary denotation.  This is always true of the sentence-level meanings, which I assume to be of type `<<t,t>,t>` following Barker.  The identity function can be thought of as an empty context of sorts.

Two more examples for type e, and type `<e,t>`, are given as well.

In [2]:
%%lamb 
||raining1|| = Raining_t # reminder: hit shift-enter to run
||raining2|| = L f_<t,t> : f(Raining_t)
||john1|| = John_e
||john2|| = L f_<e,t> : f(John_e)
||cat1|| = L x_e : Cat(x)
||cat2|| = L f_<<e,t>,t> : f(L x_e : Cat(x))

INFO (meta): Coerced guessed type for 'Cat_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Cat_t' into <e,t>, to match argument 'x_e'


$[\![\mathbf{\text{raining1}}]\!]^{}_{t} \:=\: $${Raining}_{t}$<br />
$[\![\mathbf{\text{raining2}}]\!]^{}_{\left\langle{}\left\langle{}t,t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}t,t\right\rangle{}} \: . \: {f}({Raining}_{t})$<br />
$[\![\mathbf{\text{john1}}]\!]^{}_{e} \:=\: $${John}_{e}$<br />
$[\![\mathbf{\text{john2}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: {f}({John}_{e})$<br />
$[\![\mathbf{\text{cat1}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Cat}({x})$<br />
$[\![\mathbf{\text{cat2}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {Cat}({x}))$

Mapping any ordinary denotation to its continuation is relatively straightforward at a mechanical level, at least.  `econt` below illustrates a combinator that does this for type e.

In [3]:
%%lamb
econt = L x_e : L f_<e,t> : f(x)

${econt}_{\left\langle{}e,\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}}\:=\:\lambda{} x_{e} \: . \: \lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: {f}({x})$

In [4]:
econt(john1.content).reduce_all() # we get back the content of john2

(λ f_<e,t>: f_<e,t>(John_e))

The following function generalizes this; given any type it constructs the continuization combinator for that type.  This is illustrated using `cat1`.

In [5]:
%%lamb
continuize = L x_X : L f_<X,t> : f(x)

${continuize}_{\left\langle{}X,\left\langle{}\left\langle{}X,t\right\rangle{},t\right\rangle{}\right\rangle{}}\:=\:\lambda{} x_{X} \: . \: \lambda{} f_{\left\langle{}X,t\right\rangle{}} \: . \: {f}({x})$

This function could be used for e.g. constructing a typeshift or unary composition operation (as in the Kubota and Uegaki system).  Here I will use it slightly differently, to construct a 'lexical transform' that can be applied in metalanguage definitions.  This is indicated by the =<> notation below.

In [6]:
def continuize_lex(te):
    new_te = continuize(te).reduce_all()
    return new_te

#continuize_lex(cat1.content)
lamb.parsing.eq_transforms["cont"] = continuize_lex

In [7]:
%%lamb
||cat2|| =<cont> L x_e : Cat(x)
||dance2|| =<cont> L x_e : Dance(x)
||john2|| =<cont> John_e
||the|| =<cont> L f_<e,t> : Iota x_e : f(x)

INFO (meta): Coerced guessed type for 'Cat_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Dance_t' into <e,t>, to match argument 'x_e'


$[\![\mathbf{\text{cat2}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {Cat}({x}))$<br />
$[\![\mathbf{\text{dance2}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {Dance}({x}))$<br />
$[\![\mathbf{\text{john2}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: {f}({John}_{e})$<br />
$[\![\mathbf{\text{the}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \iota{} x_{e} \: . \: {f}({x}))$

While in some cases standard FA can actually work and even produce effectively the right result, this isn't really what we want to do.  Note for example that given the above types, and regular function application, there would be no way to compose `the` with `cat`.

A continuized apply is somewhat complicated and easiest to see through working examples.  It needs to, effectively, sequence continuations.  The following example is a combinator that continuizes just the composition of a property (ordinary type `<e,t>`) with its argument. `b` is roughly the decontinuized version of the function, and `c` the decontinuized version of the argument; `abar` is the continuation argument for the whole expression.  Below this I illustrate this with `dance` and `john`, with some of the internal steps of the derivation revealed.

In [8]:
%%lamb
cfaet = L f_<<<e,t>,t>,t> : L arg_<<e,t>,t> : L abar_<t,t> : f(L b_<e,t> : arg(L c_e : abar(b(c))))

${cfaet}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{},\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},\left\langle{}\left\langle{}t,t\right\rangle{},t\right\rangle{}\right\rangle{}\right\rangle{}}\:=\:\lambda{} f_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} arg_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} abar_{\left\langle{}t,t\right\rangle{}} \: . \: {f}(\lambda{} b_{\left\langle{}e,t\right\rangle{}} \: . \: {arg}(\lambda{} c_{e} \: . \: {abar}({b}({c}))))$

In [9]:
(cfaet(dance2.content)(john2.content)).reduce_all().derivation

1. ((λ f_<<<e,t>,t>,t>: (λ arg_<<e,t>,t>: (λ abar_<t,t>: f_<<<e,t>,t>,t>((λ b_<e,t>: arg_<<e,t>,t>((λ c_e: abar_<t,t>(b_<e,t>(c_e))))))))))((λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e)))))((λ f_<e,t>: f_<e,t>(John_e)))
 2. ((λ arg_<<e,t>,t>: (λ abar_<t,t>: arg_<<e,t>,t>((λ c_e: abar_<t,t>(Dance_<e,t>(c_e)))))))((λ f_<e,t>: f_<e,t>(John_e)))    (Recursive reduction of operand 0)
 3. (λ abar_<t,t>: ((λ f_<e,t>: f_<e,t>(John_e)))((λ c_e: abar_<t,t>(Dance_<e,t>(c_e)))))    (Reduction)
 4. (λ abar_<t,t>: abar_<t,t>(Dance_<e,t>(John_e)))    (Recursive reduction of body)

To build a compositional rule along these lines we can generate a combinator like this at arbitrary values of function type and argument type.  The following combinator looks quite complicated, but the heart of it is `b(c)`.  `b` is the decontinuized version of the function `f`, and `c` is the decontinuized version of the argument `arg`.

*Note*: Barker implements application as a set of rule-to-rule (i.e. category specific) operations.  I'm not personally a fan of this style (and in any case the lambda notebook doesn't currently have the infrastructure) so I will implement things via general composition rules.  This is important to keep in mind, since the implementation here overgenerates quite a bit in consequence.  This is very similar to the Kubota and Uegaki system mentioned in the intro.

In [10]:
%%lamb
contapply = L f_Z : L arg_Z1 : L abar_Z2 : f(L b_<X,X1> : arg(L c_X : abar(b_<X,X1>(c_X))))

${contapply}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}X,X'\right\rangle{},X_{4}\right\rangle{},X_{5}\right\rangle{},\left\langle{}\left\langle{}\left\langle{}X,X'''\right\rangle{},X_{4}\right\rangle{},\left\langle{}\left\langle{}X'',X'''\right\rangle{},X_{5}\right\rangle{}\right\rangle{}\right\rangle{}}\:=\:\lambda{} f_{\left\langle{}\left\langle{}\left\langle{}X,X'\right\rangle{},X_{4}\right\rangle{},X_{5}\right\rangle{}} \: . \: \lambda{} arg_{\left\langle{}\left\langle{}X,X'''\right\rangle{},X_{4}\right\rangle{}} \: . \: \lambda{} abar_{\left\langle{}X'',X'''\right\rangle{}} \: . \: {f}(\lambda{} b_{\left\langle{}X,X'\right\rangle{}} \: . \: {arg}(\lambda{} c_{X} \: . \: {abar}({b}({c}))))$

Here are a few examples of generated combinators in action.

In [11]:
contapply(cat2.content)(john2.content)

((λ f_<<<e,t>,t>,t>: (λ arg_<<e,t>,t>: (λ abar_<t,t>: f_<<<e,t>,t>,t>((λ b_<e,t>: arg_<<e,t>,t>((λ c_e: abar_<t,t>(b_<e,t>(c_e))))))))))((λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Cat_<e,t>(x_e)))))((λ f_<e,t>: f_<e,t>(John_e)))

In [12]:
contapply(cat2.content)(john2.content).reduce_all()

(λ abar_<t,t>: abar_<t,t>(Cat_<e,t>(John_e)))

In [13]:
contapply(continuize(cat1.content))(continuize(john1.content)).reduce_all()

(λ abar_<t,t>: abar_<t,t>(Cat_<e,t>(John_e)))

In [14]:
contapply(the.content)(cat2.content)

((λ f_<<<<e,t>,e>,t>,t>: (λ arg_<<<e,t>,t>,t>: (λ abar_<e,t>: f_<<<<e,t>,e>,t>,t>((λ b_<<e,t>,e>: arg_<<<e,t>,t>,t>((λ c_<e,t>: abar_<e,t>(b_<<e,t>,e>(c_<e,t>))))))))))((λ f_<<<e,t>,e>,t>: f_<<<e,t>,e>,t>((λ f_<e,t>: (ι x_e: f_<e,t>(x_e))))))((λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Cat_<e,t>(x_e)))))

In [15]:
contapply(the.content)(cat2.content).reduce_all()

(λ abar_<e,t>: abar_<e,t>((ι x_e: Cat_<e,t>(x_e))))

Add this operation as a composition operation named `CA`.  Below this are a few examples of the rule in action.

In [16]:
system = lang.td_system.copy()
system.remove_rule("FA")
system.remove_rule("PA")
system.remove_rule("PM")
#system.add_rule(ca_op)
system.add_binary_rule(contapply, "CA")
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [17]:
(john2 * dance2).tree()

1 composition path:<br /><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{dance2}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {Dance}({x}))$</span></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{john2}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: {f}({John}_{e})$</span></div></div></div></div></div></div></div><div style="display:table-cell;vertical-align:middle;border-left:1px solid #848482;padding:0.5em"><div style="white-space:nowrap; color:blue;"><b>[<span>CA</span>]</b></div></div></div><div align="center" style="display:table-row;padding: 5px;"><div align="center" style="display:table-cell;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{[dance2 john2]}}]\!]^{}_{\left\langle{}\left\langle{}t,t\right\rangle{},t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} abar_{\left\langle{}t,t\right\rangle{}} \: . \: {abar}({Dance}({John}_{e}))$</span></div></div></div></div></div><div style="display:table-cell;"></div></div></div><br /><br />

In [18]:
%%lamb
||saw|| =<cont> L y_e : L x_e : Saw(x,y)
||mary|| =<cont> Mary_e

INFO (meta): Coerced guessed type for 'Saw_t' into <(e,e),t>, to match argument '(x_e, y_e)'


$[\![\mathbf{\text{saw}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} y_{e} \: . \: \lambda{} x_{e} \: . \: {Saw}({x}, {y}))$<br />
$[\![\mathbf{\text{mary}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: {f}({Mary}_{e})$

In [19]:
john2 * (saw * mary)

CompositionResult(results=[⟦[[saw mary] john2]⟧ = (λ abar_<t,t>: abar_<t,t>(Saw_<(e,e),t>(John_e, Mary_e)))], failures=[⟦[john2 [saw mary]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ f_<e,t>: f_<e,t>(John_e))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed))])

In [20]:
(john2 * (saw * mary)).tree()

1 composition path:<br /><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{saw}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},t\right\rangle{},t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} f_{\left\langle{}\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} y_{e} \: . \: \lambda{} x_{e} \: . \: {Saw}({x}, {y}))$</span></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{mary}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: {f}({Mary}_{e})$</span></div></div></div></div></div></div></div><div style="display:table-cell;vertical-align:middle;border-left:1px solid #848482;padding:0.5em"><div style="white-space:nowrap; color:blue;"><b>[<span>CA</span>]</b></div></div></div><div align="center" style="display:table-row;padding: 5px;"><div align="center" style="display:table-cell;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{[saw mary]}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} abar_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {abar}(\lambda{} x_{e} \: . \: {Saw}({x}, {Mary}_{e}))$</span></div></div></div></div></div><div style="display:table-cell;"></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{john2}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} f_{\

### Quantification

At this point it is time to turn to quantifiers.  Items like `everyone` have continuized types, but are not generated by continuizing an ordinary meaning.  Rather, they are written as continuized types that manipulate their continuations.  In fact, their standard GQ entry is their continuized entry.  For comparison, a continuized version of ordinary "everyone" is given as `everyone0`.  (All of these ignore animacy.)  While these two entries for `everyone` get the same result in subject position (shown below), they do it in different ways.

In [21]:
%%lamb
||someone|| = L xbar_<e,t> : Exists x_e : xbar(x)
||everyone|| = L xbar_<e,t> : Forall x_e : xbar(x)
||everyone0|| =<cont> L f_<e,t> :  Forall x_e : f(x)

$[\![\mathbf{\text{someone}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} xbar_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: {xbar}({x})$<br />
$[\![\mathbf{\text{everyone}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} xbar_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: {xbar}({x})$<br />
$[\![\mathbf{\text{everyone0}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: {f}({x}))$

In [22]:
everyone * (saw * mary)

CompositionResult(results=[⟦[[saw mary] everyone]⟧ = (λ abar_<t,t>: (Forall x_e: abar_<t,t>(Saw_<(e,e),t>(x_e, Mary_e))))], failures=[⟦[everyone [saw mary]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Forall x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed))])

In [23]:
everyone0 * (saw * mary)

CompositionResult(results=[⟦[everyone0 [saw mary]]⟧ = (λ abar_<t,t>: abar_<t,t>((Forall x_e: Saw_<(e,e),t>(x_e, Mary_e))))], failures=[⟦[[saw mary] everyone0]⟧ = Type mismatch: '((λ f_<<<e,t>,t>,t>: (λ arg_<<e,I830>,t>: (λ abar_<I828,I830>: f_<<<e,t>,t>,t>((λ b_<e,t>: arg_<<e,I830>,t>((λ c_e: abar_<t,I830>(b_<e,t>(c_e))))))))))((λ abar_<<e,t>,t>: abar_<<e,t>,t>((λ x_e: Saw_<(e,e),t>(x_e, Mary_e)))))'/<<<e,I830>,t>,<<I828,I830>,t>> and '(λ f_<<<e,t>,t>,t>: f_<<<e,t>,t>,t>((λ f_<e,t>: (Forall x_e: f_<e,t>(x_e)))))'/<<<<e,t>,t>,t>,t> conflict (mode: Function argument combination (unification failed))])

`everyone0` will not work in object position (as in standard approaches cf. Heim and Kratzer), but the Barker's versions will, effectively for "free" (once the infrastructure is accepted).  The first example shows what happens with the continuized ordinary generalized quantifer; the resulting errors are generated inside the two possible continuized apply combinators.  The other examples demonstrate Barker's quantifiers in object position.

In [24]:
(saw * everyone0)

CompositionResult(results=[], failures=[⟦[saw everyone0]⟧ = Type mismatch: '((λ f_<<<e,<e,t>>,t>,t>: (λ arg_<<e,I855>,t>: (λ abar_<I853,I855>: f_<<<e,<e,t>>,t>,t>((λ b_<e,<e,t>>: arg_<<e,I855>,t>((λ c_e: abar_<<e,t>,I855>(b_<e,<e,t>>(c_e))))))))))((λ f_<<e,<e,t>>,t>: f_<<e,<e,t>>,t>((λ y_e: (λ x_e: Saw_<(e,e),t>(x_e, y_e))))))'/<<<e,I855>,t>,<<I853,I855>,t>> and '(λ f_<<<e,t>,t>,t>: f_<<<e,t>,t>,t>((λ f_<e,t>: (Forall x_e: f_<e,t>(x_e)))))'/<<<<e,t>,t>,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[everyone0 saw]⟧ = Type mismatch: '((λ f_<<<<e,t>,t>,t>,t>: (λ arg_<<<e,t>,I880>,t>: (λ abar_<I878,I880>: f_<<<<e,t>,t>,t>,t>((λ b_<<e,t>,t>: arg_<<<e,t>,I880>,t>((λ c_<e,t>: abar_<t,I880>(b_<<e,t>,t>(c_<e,t>))))))))))((λ f_<<<e,t>,t>,t>: f_<<<e,t>,t>,t>((λ f_<e,t>: (Forall x_e: f_<e,t>(x_e))))))'/<<<<e,t>,I880>,t>,<<I878,I880>,t>> and '(λ f_<<e,<e,t>>,t>: f_<<e,<e,t>>,t>((λ y_e: (λ x_e: Saw_<(e,e),t>(x_e, y_e)))))'/<<<e,<e,t>>,t>,t> conflict (mode: Function argument combination (unification failed))])

In [25]:
mary * (saw * everyone)

CompositionResult(results=[⟦[[saw everyone] mary]⟧ = (λ abar_<t,t>: (Forall x_e: abar_<t,t>(Saw_<(e,e),t>(Mary_e, x_e))))], failures=[⟦[mary [saw everyone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ f_<e,t>: f_<e,t>(Mary_e))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed))])

In [26]:
everyone * (saw * someone)

CompositionResult(results=[⟦[[saw someone] everyone]⟧ = (λ abar_<t,t>: (Exists x_e: (Forall x1_e: abar_<t,t>(Saw_<(e,e),t>(x1_e, x_e)))))], failures=[⟦[everyone [saw someone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Forall x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed))])

Multiple quantifiers work as well, shown above.  This generates inverse scope; we will attend to surface scope shortly.  I started with inverse scope because Barker does, but I'm not aware of any real significance to this choice.

To get surface scope, we need a second version of application which prioritizes continuations differently.

In [27]:
%%lamb
contapply2 = L f_Z : L arg_Z1 : L abar_Z2 : arg(L c_X : f(L b_<X,X1>: abar(b_<X,X1>(c_X))))

${contapply2}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}X,X'\right\rangle{},X'''\right\rangle{},X_{4}\right\rangle{},\left\langle{}\left\langle{}\left\langle{}X,X_{4}\right\rangle{},X_{5}\right\rangle{},\left\langle{}\left\langle{}X'',X'''\right\rangle{},X_{5}\right\rangle{}\right\rangle{}\right\rangle{}}\:=\:\lambda{} f_{\left\langle{}\left\langle{}\left\langle{}X,X'\right\rangle{},X'''\right\rangle{},X_{4}\right\rangle{}} \: . \: \lambda{} arg_{\left\langle{}\left\langle{}X,X_{4}\right\rangle{},X_{5}\right\rangle{}} \: . \: \lambda{} abar_{\left\langle{}X'',X'''\right\rangle{}} \: . \: {arg}(\lambda{} c_{X} \: . \: {f}(\lambda{} b_{\left\langle{}X,X'\right\rangle{}} \: . \: {abar}({b}({c}))))$

In [28]:
system = lang.td_system.copy()
system.remove_rule("FA")
system.remove_rule("PA")
system.remove_rule("PM")
system.add_binary_rule(contapply, "CA")
system.add_binary_rule(contapply2, "CA2")
lang.set_system(system)
system

Composition system: H&K simple (copy)

And on to the interesting stuff.  In general each of these rules can apply to any continuized type, so now we do overgenerate the same result; the lambda notebook will collapse these because they are equivalent.  (Barker overgenerates less, because of his use of rule-to-rule translation.)  All the results are right, so this isn't really a problem.  When there are multiple quantifiers, we generate both readings.

In [29]:
everyone * (saw * mary)

CompositionResult(results=[⟦[[saw mary] everyone]⟧ = (λ abar_<t,t>: (Forall x_e: abar_<t,t>(Saw_<(e,e),t>(x_e, Mary_e))))], failures=[⟦[everyone [saw mary]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Forall x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[everyone [saw mary]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Forall x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed))])

In [30]:
everyone * (saw * someone)

CompositionResult(results=[⟦[[saw someone] everyone]⟧ = (λ abar_<t,t>: (Exists x_e: (Forall x1_e: abar_<t,t>(Saw_<(e,e),t>(x1_e, x_e))))), ⟦[[saw someone] everyone]⟧ = (λ abar_<t,t>: (Forall x_e: (Exists x1_e: abar_<t,t>(Saw_<(e,e),t>(x_e, x1_e)))))], failures=[⟦[everyone [saw someone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Forall x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[everyone [saw someone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Forall x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed))])

In [31]:
(someone * (saw * everyone))

CompositionResult(results=[⟦[[saw everyone] someone]⟧ = (λ abar_<t,t>: (Forall x_e: (Exists x1_e: abar_<t,t>(Saw_<(e,e),t>(x1_e, x_e))))), ⟦[[saw everyone] someone]⟧ = (λ abar_<t,t>: (Exists x_e: (Forall x1_e: abar_<t,t>(Saw_<(e,e),t>(x_e, x1_e)))))], failures=[⟦[someone [saw everyone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Exists x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[someone [saw everyone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Exists x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed))])

If you're wondering about the overgeneration, this can be inspected in a number of ways, but the easiest here is with a tree.  The tree will show alternative composition operations that lead to the same result.  For the inverse scope reading, either CA or CA2 can apply to compose the verb with the object:

In [32]:
(someone * (saw * everyone))[1].tree()

What about quantificational determiners?  Barker presents two treatments.  In the text, regular FA is allowed to combine a determiner with its complement (in a rule-to-rule fashion).  In the appendix, a different general treatment not requiring FA, but using choice functions, is presented.  For the moment I stick to the text version, and allow FA as a general possibility (so I will overgenerate more than Barker did, a la Kubota and Uegaki).  Later we'll need the choice function version.

In [33]:
%%lamb
||every|| = L pbar_<<<e,t>,t>,t> : L xbar_<e,t> : pbar(L f_<e,t> : Forall x_e : (f(x) >> xbar(x)))

$[\![\mathbf{\text{every}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} pbar_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} xbar_{\left\langle{}e,t\right\rangle{}} \: . \: {pbar}(\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: ({f}({x}) \rightarrow{} {xbar}({x})))$

In [34]:
system = lang.td_system.copy()
#system.remove_rule("FA")
system.remove_rule("PA")
system.remove_rule("PM")
system.add_binary_rule(contapply, "CA")
system.add_binary_rule(contapply2, "CA2")
lang.set_system(system)

def tfilter_fun(i):
    return (i.type == lang.tp("<<t,t>,t>"))

tfilter = lang.CRFilter("S-filter", tfilter_fun)

The latter part of the above box prunes down on a bit of overgeneration; any derivation that does not result in a sentence type (e.g. `<<t,t>,t>` in a continuized setting) is eliminated.

Another form of overgeneration is that we can see that the CA rule can also apply in D-N combination, though the derivation resulting from this choice won't converge so it doesn't especially matter.

In [35]:
(every * cat2).tree()

2 composition paths:<br />
Path [0]:<br />
<div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{every}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} pbar_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} xbar_{\left\langle{}e,t\right\rangle{}} \: . \: {pbar}(\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: ({f}({x}) \rightarrow{} {xbar}({x})))$</span></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{cat2}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {Cat}({x}))$</span></div></div></div></div></div></div></div><div style="display:table-cell;vertical-align:middle;border-left:1px solid #848482;padding:0.5em"><div style="white-space:nowrap; color:blue;"><b>[<span>FA</span>]</b></div></div></div><div align="center" style="display:table-row;padding: 5px;"><div align="center" style="display:table-cell;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{[every cat2]}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} xbar_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} x_{e} \: . \: ({Cat}({x}) \rightarrow{} {xbar}({x}))$</span></div></div></div></div></div><div style="display:table-cell;"></div></div></div><br /><br />Path [1]:<br />
<div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{every}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{},\left\langle{}\left\langle{}e,t\right\ra

In [36]:
r = (every * cat2) * (saw * someone)
tfilter(r)

CompositionResult(results=[⟦[[saw someone] [every cat2]]⟧ = (λ abar_<t,t>: (Exists x_e: (Forall x1_e: (Cat_<e,t>(x1_e) >> abar_<t,t>(Saw_<(e,e),t>(x1_e, x_e)))))), ⟦[[saw someone] [every cat2]]⟧ = (λ abar_<t,t>: (Forall x_e: (Cat_<e,t>(x_e) >> (Exists x1_e: abar_<t,t>(Saw_<(e,e),t>(x_e, x1_e))))))], failures=[⟦[[every cat2] [saw someone]]⟧ = Type mismatch: '⟦[every cat2]⟧ = (λ xbar_<e,t>: (Forall x_e: (Cat_<e,t>(x_e) >> xbar_<e,t>(x_e))))'/<<e,t>,t> and '⟦[saw someone]⟧ = (λ abar_<<e,t>,t>: (Exists x_e: abar_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e)))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[[every cat2] [saw someone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Forall x_e: (Cat_<e,t>(x_e) >> xbar_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[every cat2] [saw someone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Forall x_e: (Cat_<e,t>(x_e) >> xbar_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[every cat2] [saw someone]]⟧ = Type mismatch: '⟦[every cat2]⟧ = (λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>((Forall x_e: (Cat_<e,t>(x_e) >> xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> and '⟦[saw someone]⟧ = (λ abar_<<e,t>,t>: (Exists x_e: abar_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e)))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[[saw someone] [every cat2]]⟧ = Type mismatch: '⟦[saw someone]⟧ = (λ abar_<<e,t>,t>: (Exists x_e: abar_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e)))))'/<<<e,t>,t>,t> and '⟦[every cat2]⟧ = (λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>((Forall x_e: (Cat_<e,t>(x_e) >> xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function Application), ⟦[[every cat2] [saw someone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>((Forall x_e: (Cat_<e,t>(x_e) >> xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[[saw someone] [every cat2]]⟧ = Type mismatch: '((λ f_<<<e,t>,t>,t>: (λ arg_<<e,I2604>,t>: (λ abar_<I2602,I2604>: f_<<<e,t>,t>,t>((λ b_<e,t>: arg_<<e,I2604>,t>((λ c_e: abar_<t,I2604>(b_<e,t>(c_e))))))))))((λ abar_<<e,t>,t>: (Exists x_e: abar_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e))))))'/<<<e,I2604>,t>,<<I2602,I2604>,t>> and '(λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>((Forall x_e: (Cat_<e,t>(x_e) >> xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[[every cat2] [saw someone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>((Forall x_e: (Cat_<e,t>(x_e) >> xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[[saw someone] [every cat2]]⟧ = Type mismatch: '((λ f_<<<e,t>,t>,t>: (λ arg_<<e,t>,I2631>: (λ abar_<I2634,t>: arg_<<e,t>,I2631>((λ c_e: f_<<<e,t>,t>,t>((λ b_<e,t>: abar_<t,t>(b_<e,t>(c_e))))))))))((λ abar_<<e,t>,t>: (Exists x_e: abar_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e))))))'/<<<e,t>,I2631>,<<I2634,t>,I2631>> and '(λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>((Forall x_e: (Cat_<e,t>(x_e) >> xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function ar

In [37]:
r[1].tree()

### Scope bounding

In order to make tensed S nodes a scope island, Barker provides a different composition rule for S$\rightarrow$NP VP nodes that blocks continuation passing.  In a setting with generalized composition rules, this needs to be done a bit differently.  One can define an operator that performs the same function as this rule, and this operator might be lexically instantiated by e.g. `that`.  I've named this operator `Disrupt`:

In [38]:
%%lamb
||Disrupt|| = L s_<<t,t>,t> : L abar_<t,t> : abar(s(L p_t : p))

$[\![\mathbf{\text{Disrupt}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}t,t\right\rangle{},t\right\rangle{},\left\langle{}\left\langle{}t,t\right\rangle{},t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} s_{\left\langle{}\left\langle{}t,t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} abar_{\left\langle{}t,t\right\rangle{}} \: . \: {abar}({s}(\lambda{} p_{t} \: . \: {p}))$

It is actually somewhat hard to find examples that really use this in an extensional setting with no binding/traces; Barker provides conjunction.  I'm not (currently) treating conjunction in this fragment so we must search for something a little different.  One relevant case is the simplest analysis of embedding predicates like `it is false that` as negation.  Empirically, quantifiers cannot scope over this, though they can scope over ordinary negation.

1. It is not true that some student left. ($\neg >> \exists$)
2. Some student didn't leave.  ($\exists >> \neg$, or marginally $\neg >> \exists$)

In [39]:
%%lamb
||iift|| =<cont> L p_t : ~p 

$[\![\mathbf{\text{iift}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}t,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}\left\langle{}t,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} p_{t} \: . \: \neg{} {p})$

We can first see that Disrupt has a non-substantive but visible effect on simple quantificational sentences.  In one case, the quantifier scopes under the continuation, in the other case, over.

In [40]:
tfilter(Disrupt * (someone * dance2))

CompositionResult(results=[⟦[Disrupt [dance2 someone]]⟧ = (λ abar_<t,t>: abar_<t,t>((Exists x_e: Dance_<e,t>(x_e))))], failures=[⟦[Disrupt [dance2 someone]]⟧ = Type mismatch: '⟦Disrupt⟧ = (λ s_<<t,t>,t>: (λ abar_<t,t>: abar_<t,t>(s_<<t,t>,t>((λ p_t: p_t)))))'/<<<t,t>,t>,<<t,t>,t>> and '⟦[dance2 someone]⟧ = (Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function Application), ⟦[[dance2 someone] Disrupt]⟧ = Type mismatch: '⟦[dance2 someone]⟧ = (Exists x_e: Dance_<e,t>(x_e))'/t and '⟦Disrupt⟧ = (λ s_<<t,t>,t>: (λ abar_<t,t>: abar_<t,t>(s_<<t,t>,t>((λ p_t: p_t)))))'/<<<t,t>,t>,<<t,t>,t>> conflict (mode: Function Application), ⟦[Disrupt [dance2 someone]]⟧ = Type mismatch: '((λ f_<<<t,t>,t>,<<t,t>,t>>: (λ arg_<<t,I2793>,t>: (λ abar_<I2791,I2793>: f_<<<t,t>,t>,<<t,t>,t>>((λ b_<t,t>: arg_<<t,I2793>,t>((λ c_t: abar_<t,I2793>(b_<t,t>(c_t))))))))))((λ s_<<t,t>,t>: (λ abar_<t,t>: abar_<t,t>(s_<<t,t>,t>((λ p_t: p_t))))))'/<<<t,I2793>,t>,<<I2791,I2793>,<<t,t>,t>>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] Disrupt]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[Disrupt [dance2 someone]]⟧ = Type mismatch: '((λ f_<<<t,t>,t>,<<t,t>,t>>: (λ arg_<<t,<<t,t>,t>>,I2830>: (λ abar_<I2833,t>: arg_<<t,<<t,t>,t>>,I2830>((λ c_t: f_<<<t,t>,t>,<<t,t>,t>>((λ b_<t,t>: abar_<t,t>(b_<t,t>(c_t))))))))))((λ s_<<t,t>,t>: (λ abar_<t,t>: abar_<t,t>(s_<<t,t>,t>((λ p_t: p_t))))))'/<<<t,<<t,t>,t>>,I2830>,<<I2833,t>,I2830>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] Disrupt]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] Disrupt]⟧ = Type mismatch: '⟦[dance2 someone]⟧ = (λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))'/<<t,t>,t> and '⟦Disrupt⟧ = (λ s_<<t,t>,t>: (λ abar_<t,t>: abar_<t,t>(s_<<t,t>,t>((λ p_t: p_t)))))'/<<<t,t>,t>,<<t,t>,t>> conflict (mode: Function Application), ⟦[[dance2 someone] Disrupt]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))'/<<t,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[Disrupt [dance2 someone]]⟧ = Type mismatch: '((λ f_<<<t,t>,t>,<<t,t>,t>>: (λ arg_<<t,<<t,t>,t>>,I2948>: (λ abar_<I2951,t>: arg_<<t,<<t,t>,t>>,I2948>((λ c_t: f_<<<t,t>,t>,<<t,t>,t>>((λ b_<t,t>: abar_<t,t>(b_<t,t>(c_t))))))))))((λ s_<<t,t>,t>: (λ abar_<t,t>: abar_<t,t>(s_<<t,t>,t>((λ p_t: p_t))))))'/<<<t,<<t,t>,t>>,I2948>,<<I2951,t>,I2948>> and '(λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))'/<<t,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] Disrupt]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))'/<<t,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[Disrupt [dance2 someone]]⟧ = (λ abar_<t,t>: (λ abar1_<t,t>: abar1_<t,t>((Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))))])

In [41]:
tfilter(someone * dance2)

CompositionResult(results=[⟦[dance2 someone]⟧ = (λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))], failures=[⟦[someone dance2]⟧ = Type mismatch: '⟦someone⟧ = (λ xbar_<e,t>: (Exists x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> and '⟦dance2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[someone dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Exists x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[someone dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Exists x_e: xbar_<e,t>(x_e)))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[dance2 someone]⟧ = (Exists x_e: Dance_<e,t>(x_e))])

This now does something interesting when `iift` (_it is false that_) composes with or without Disrupt:

In [42]:
tfilter(iift * (someone * dance2))

CompositionResult(results=[⟦[iift [dance2 someone]]⟧ = (λ abar_<t,t>: (Exists x_e: abar_<t,t>(~Dance_<e,t>(x_e))))], failures=[⟦[iift [dance2 someone]]⟧ = Type mismatch: '⟦iift⟧ = (λ f_<<t,t>,t>: f_<<t,t>,t>((λ p_t: ~p_t)))'/<<<t,t>,t>,t> and '⟦[dance2 someone]⟧ = (Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function Application), ⟦[[dance2 someone] iift]⟧ = Type mismatch: '⟦[dance2 someone]⟧ = (Exists x_e: Dance_<e,t>(x_e))'/t and '⟦iift⟧ = (λ f_<<t,t>,t>: f_<<t,t>,t>((λ p_t: ~p_t)))'/<<<t,t>,t>,t> conflict (mode: Function Application), ⟦[iift [dance2 someone]]⟧ = Type mismatch: '((λ f_<<<t,t>,t>,t>: (λ arg_<<t,I3242>,t>: (λ abar_<I3240,I3242>: f_<<<t,t>,t>,t>((λ b_<t,t>: arg_<<t,I3242>,t>((λ c_t: abar_<t,I3242>(b_<t,t>(c_t))))))))))((λ f_<<t,t>,t>: f_<<t,t>,t>((λ p_t: ~p_t))))'/<<<t,I3242>,t>,<<I3240,I3242>,t>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] iift]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[iift [dance2 someone]]⟧ = Type mismatch: '((λ f_<<<t,t>,t>,t>: (λ arg_<<t,t>,I3269>: (λ abar_<I3272,t>: arg_<<t,t>,I3269>((λ c_t: f_<<<t,t>,t>,t>((λ b_<t,t>: abar_<t,t>(b_<t,t>(c_t))))))))))((λ f_<<t,t>,t>: f_<<t,t>,t>((λ p_t: ~p_t))))'/<<<t,t>,I3269>,<<I3272,t>,I3269>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] iift]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] iift]⟧ = Type mismatch: '⟦[dance2 someone]⟧ = (λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))'/<<t,t>,t> and '⟦iift⟧ = (λ f_<<t,t>,t>: f_<<t,t>,t>((λ p_t: ~p_t)))'/<<<t,t>,t>,t> conflict (mode: Function Application), ⟦[[dance2 someone] iift]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))'/<<t,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] iift]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))'/<<t,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[iift [dance2 someone]]⟧ = (Exists x_e: ~Dance_<e,t>(x_e))])

In [43]:
r2 = tfilter(iift * (Disrupt * (someone * dance2)))
r2

CompositionResult(results=[⟦[iift [Disrupt [dance2 someone]]]⟧ = (λ abar_<t,t>: abar_<t,t>(~(Exists x_e: Dance_<e,t>(x_e))))], failures=[⟦[[Disrupt [dance2 someone]] iift]⟧ = Type mismatch: '⟦[Disrupt [dance2 someone]]⟧ = (λ abar_<t,t>: abar_<t,t>((Exists x_e: Dance_<e,t>(x_e))))'/<<t,t>,t> and '⟦iift⟧ = (λ f_<<t,t>,t>: f_<<t,t>,t>((λ p_t: ~p_t)))'/<<<t,t>,t>,t> conflict (mode: Function Application), ⟦[[Disrupt [dance2 someone]] iift]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: abar_<t,t>((Exists x_e: Dance_<e,t>(x_e))))'/<<t,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[Disrupt [dance2 someone]] iift]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: abar_<t,t>((Exists x_e: Dance_<e,t>(x_e))))'/<<t,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[iift [Disrupt [dance2 someone]]]⟧ = Type mismatch: '⟦iift⟧ = (λ f_<<t,t>,t>: f_<<t,t>,t>((λ p_t: ~p_t)))'/<<<t,t>,t>,t> and '⟦[Disrupt [dance2 someone]]⟧ = (λ abar_<t,t>: (λ abar1_<t,t>: abar1_<t,t>((Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))))'/<<t,t>,<<t,t>,t>> conflict (mode: Function Application), ⟦[[Disrupt [dance2 someone]] iift]⟧ = Type mismatch: '⟦[Disrupt [dance2 someone]]⟧ = (λ abar_<t,t>: (λ abar1_<t,t>: abar1_<t,t>((Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))))'/<<t,t>,<<t,t>,t>> and '⟦iift⟧ = (λ f_<<t,t>,t>: f_<<t,t>,t>((λ p_t: ~p_t)))'/<<<t,t>,t>,t> conflict (mode: Function Application), ⟦[iift [Disrupt [dance2 someone]]]⟧ = Type mismatch: '((λ f_<<<t,t>,t>,t>: (λ arg_<<t,I3868>,t>: (λ abar_<I3866,I3868>: f_<<<t,t>,t>,t>((λ b_<t,t>: arg_<<t,I3868>,t>((λ c_t: abar_<t,I3868>(b_<t,t>(c_t))))))))))((λ f_<<t,t>,t>: f_<<t,t>,t>((λ p_t: ~p_t))))'/<<<t,I3868>,t>,<<I3866,I3868>,t>> and '(λ abar_<t,t>: (λ abar1_<t,t>: abar1_<t,t>((Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))))'/<<t,t>,<<t,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[[Disrupt [dance2 someone]] iift]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (λ abar1_<t,t>: abar1_<t,t>((Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))))'/<<t,t>,<<t,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[[Disrupt [dance2 someone]] iift]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (λ abar1_<t,t>: abar1_<t,t>((Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))))'/<<t,t>,<<t,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[iift [Disrupt [dance2 someone]]]⟧ = ~(Exists x_e: Dance_<e,t>(x_e)), ⟦[iift [Disrupt [dance2 someone]]]⟧ = (λ abar_<t,t>: (λ abar1_<t,t>: abar1_<t,t>((Exists x_e: abar_<t,t>(~Dance_<e,t>(x_e))))))])

Negation in these derivations is scopally inert, and quantifiers scope over it.  In fact this reveals an interesting property of this system: whatever items are scopally potent must scope over whatever material is scopally inert, up to operators like Disrupt or other scopal elements.

If you think about scope as a form of '_projective meaning_', then this means that in a framework like the present with only _at-issue_ meaning as the other choice, scopal elements project maximally.  This fact was exploited by Kubota and Uegaki in using continuations for Potts-style Conventional Implicature (CI) content, which should project maximally.  It is worth noting, however, that as far as I can see scope is _not_ CI content as such, and we would certainly not want scopal ambiguities to interact with projective meanings in the way that combining the two uses of continuations would involve.  (At the least, tensed finite clauses are not generally a scope island for CI content.)  In summary, it is currently unclear (to me) that continuations can be used as a _general_ mechanism for projective meaning displacement in a compositional semantics, because there are different empirical properties for different types of projective meaning; I don't yet see how it could account for both quantifier scope and CI projection at the same time.

Moving on, one can generate a non-scopally-inert negation by using a similar trick to what was performed on the quantifiers, i.e. applying the content of the item after the continuation.  This is shown below:

In [44]:
%%lamb
||sneg|| = L f_<<t,t>,t> : ~ f(L p_t : p)

$[\![\mathbf{\text{sneg}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}t,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}\left\langle{}t,t\right\rangle{},t\right\rangle{}} \: . \: \neg{} {f}(\lambda{} p_{t} \: . \: {p})$

Now negation of this form must take widest form w.r.t. scopally inert elements, but will scope ambiguously w.r.t. scopal elements.

In [45]:
tfilter(sneg * (someone * dance2))

CompositionResult(results=[⟦[sneg [dance2 someone]]⟧ = (λ abar_<t,t>: ~(Exists x_e: abar_<t,t>(Dance_<e,t>(x_e)))), ⟦[sneg [dance2 someone]]⟧ = (λ abar_<t,t>: (Exists x_e: ~abar_<t,t>(Dance_<e,t>(x_e))))], failures=[⟦[sneg [dance2 someone]]⟧ = Type mismatch: '⟦sneg⟧ = (λ f_<<t,t>,t>: ~f_<<t,t>,t>((λ p_t: p_t)))'/<<<t,t>,t>,t> and '⟦[dance2 someone]⟧ = (Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function Application), ⟦[[dance2 someone] sneg]⟧ = Type mismatch: '⟦[dance2 someone]⟧ = (Exists x_e: Dance_<e,t>(x_e))'/t and '⟦sneg⟧ = (λ f_<<t,t>,t>: ~f_<<t,t>,t>((λ p_t: p_t)))'/<<<t,t>,t>,t> conflict (mode: Function Application), ⟦[sneg [dance2 someone]]⟧ = Type mismatch: '((λ f_<<<t,t>,t>,t>: (λ arg_<<t,I4084>,t>: (λ abar_<I4082,I4084>: f_<<<t,t>,t>,t>((λ b_<t,t>: arg_<<t,I4084>,t>((λ c_t: abar_<t,I4084>(b_<t,t>(c_t))))))))))((λ f_<<t,t>,t>: ~f_<<t,t>,t>((λ p_t: p_t))))'/<<<t,I4084>,t>,<<I4082,I4084>,t>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] sneg]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[sneg [dance2 someone]]⟧ = Type mismatch: '((λ f_<<<t,t>,t>,t>: (λ arg_<<t,t>,I4121>: (λ abar_<I4124,t>: arg_<<t,t>,I4121>((λ c_t: f_<<<t,t>,t>,t>((λ b_<t,t>: abar_<t,t>(b_<t,t>(c_t))))))))))((λ f_<<t,t>,t>: ~f_<<t,t>,t>((λ p_t: p_t))))'/<<<t,t>,I4121>,<<I4124,t>,I4121>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] sneg]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(Exists x_e: Dance_<e,t>(x_e))'/t conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] sneg]⟧ = Type mismatch: '⟦[dance2 someone]⟧ = (λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))'/<<t,t>,t> and '⟦sneg⟧ = (λ f_<<t,t>,t>: ~f_<<t,t>,t>((λ p_t: p_t)))'/<<<t,t>,t>,t> conflict (mode: Function Application), ⟦[[dance2 someone] sneg]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))'/<<t,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[dance2 someone] sneg]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (Exists x_e: abar_<t,t>(Dance_<e,t>(x_e))))'/<<t,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[sneg [dance2 someone]]⟧ = ~(Exists x_e: Dance_<e,t>(x_e))])

### Recursive DPs and inverse linking

The LF analysis of scope has in general had a hard time with the scope of DPs within DPs.  First, in many cases the judgments aren't very clear.  But to the extent that they are clear, one must block some of the readings.  Barker's proposal is that the possible readings just fall out of the architecture of continuation-based scope.  Let's see how this works.

In [46]:
%%lamb
||no|| = L pbar_<<<e,t>,t>,t> : L xbar_<e,t> : pbar(L f_<e,t> : (~ (Exists x_e : (f(x) & xbar(x)))))
||a|| = L pbar_<<<e,t>,t>,t> : L xbar_<e,t> : pbar(L f_<e,t> : (Exists x_e : (f(x) & xbar(x))))
||fromP|| =<cont> L x_e : L f_<e,t> : L y_e : f(y) & From(y,x)
||france|| =<cont> France_e
||fcountry|| =<cont> L x_e : ForeignCountry(x)

INFO (meta): Coerced guessed type for 'From_t' into <(e,e),t>, to match argument '(y_e, x_e)'
INFO (meta): Coerced guessed type for 'ForeignCountry_t' into <e,t>, to match argument 'x_e'


$[\![\mathbf{\text{no}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} pbar_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} xbar_{\left\langle{}e,t\right\rangle{}} \: . \: {pbar}(\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \neg{} \exists{} x_{e} \: . \: ({f}({x}) \wedge{} {xbar}({x})))$<br />
$[\![\mathbf{\text{a}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{},\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} pbar_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} xbar_{\left\langle{}e,t\right\rangle{}} \: . \: {pbar}(\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: ({f}({x}) \wedge{} {xbar}({x})))$<br />
$[\![\mathbf{\text{fromP}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,\left\langle{}\left\langle{}e,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}\left\langle{}e,\left\langle{}\left\langle{}e,t\right\rangle{},\left\langle{}e,t\right\rangle{}\right\rangle{}\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: \lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} y_{e} \: . \: ({f}({y}) \wedge{} {From}({y}, {x})))$<br />
$[\![\mathbf{\text{france}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: {f}({France}_{e})$<br />
$[\![\mathbf{\text{fcountry}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {ForeignCountry}({x}))$

Just as a sanity check, something like "no cat from france danced" should compose as is, and generate one scoping.  This works!

In [47]:
tfilter((no * (cat2 * (fromP * france))) * dance2)

CompositionResult(results=[⟦[dance2 [no [[fromP france] cat2]]]⟧ = (λ abar_<t,t>: ~(Exists x_e: ((Cat_<e,t>(x_e) & From_<(e,e),t>(x_e, France_e)) & abar_<t,t>(Dance_<e,t>(x_e)))))], failures=[⟦[[no [[fromP france] cat2]] dance2]⟧ = Type mismatch: '⟦[no [[fromP france] cat2]]⟧ = (λ xbar_<e,t>: ~(Exists x_e: ((Cat_<e,t>(x_e) & From_<(e,e),t>(x_e, France_e)) & xbar_<e,t>(x_e))))'/<<e,t>,t> and '⟦dance2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[[no [[fromP france] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: ~(Exists x_e: ((Cat_<e,t>(x_e) & From_<(e,e),t>(x_e, France_e)) & xbar_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP france] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: ~(Exists x_e: ((Cat_<e,t>(x_e) & From_<(e,e),t>(x_e, France_e)) & xbar_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP france] cat2]] dance2]⟧ = Type mismatch: '⟦[no [[fromP france] cat2]]⟧ = (λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>(~(Exists x_e: ((Cat_<e,t>(x_e) & From_<(e,e),t>(x_e, France_e)) & xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> and '⟦dance2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[dance2 [no [[fromP france] cat2]]]⟧ = Type mismatch: '⟦dance2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e))))'/<<<e,t>,t>,t> and '⟦[no [[fromP france] cat2]]⟧ = (λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>(~(Exists x_e: ((Cat_<e,t>(x_e) & From_<(e,e),t>(x_e, France_e)) & xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function Application), ⟦[[no [[fromP france] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>(~(Exists x_e: ((Cat_<e,t>(x_e) & From_<(e,e),t>(x_e, France_e)) & xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[dance2 [no [[fromP france] cat2]]]⟧ = Type mismatch: '((λ f_<<<e,t>,t>,t>: (λ arg_<<e,I4884>,t>: (λ abar_<I4882,I4884>: f_<<<e,t>,t>,t>((λ b_<e,t>: arg_<<e,I4884>,t>((λ c_e: abar_<t,I4884>(b_<e,t>(c_e))))))))))((λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e)))))'/<<<e,I4884>,t>,<<I4882,I4884>,t>> and '(λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>(~(Exists x_e: ((Cat_<e,t>(x_e) & From_<(e,e),t>(x_e, France_e)) & xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP france] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>(~(Exists x_e: ((Cat_<e,t>(x_e) & From_<(e,e),t>(x_e, France_e)) & xbar_<e,t>(x_e))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[dance2 [no [[fromP france] cat2]]]⟧ = Type mismatch: '((λ f_<<<e,t>,t>,t>: (λ arg_<<e,t>,I4911>: (λ abar_<I4914,t>: arg_<<e,t>,I4911>((λ c_e: f_<<<e,t>,t>,t>((λ b_<e,t>: abar_<t,t>(b_<e,t>(c_e))))))))))((λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e)))))'/<<<e,t>,I4911>,<<I4914,t>,I4911>> and '(λ abar_<t,t>: (λ xbar_<e,t>: abar_<t,t>(~(Exist

Now the real test: what happens with

1. No cat from a foreign country danced.
2. No cat from a foreign country saw someone.



In [48]:
r = tfilter((no * (cat2 * (fromP * (a * fcountry)))) * dance2)
r

CompositionResult(results=[⟦[dance2 [no [[fromP [a fcountry]] cat2]]]⟧ = (λ abar_<t,t>: (Exists x_e: (ForeignCountry_<e,t>(x_e) & ~(Exists x1_e: ((Cat_<e,t>(x1_e) & From_<(e,e),t>(x1_e, x_e)) & abar_<t,t>(Dance_<e,t>(x1_e)))))))], failures=[⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '⟦[no [[fromP [a fcountry]] cat2]]⟧ = (λ xbar_<e,t>: (Exists x_e: (ForeignCountry_<e,t>(x_e) & ~(Exists x1_e: ((Cat_<e,t>(x1_e) & From_<(e,e),t>(x1_e, x_e)) & xbar_<e,t>(x1_e))))))'/<<e,t>,t> and '⟦dance2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Exists x_e: (ForeignCountry_<e,t>(x_e) & ~(Exists x1_e: ((Cat_<e,t>(x1_e) & From_<(e,e),t>(x1_e, x_e)) & xbar_<e,t>(x1_e))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ xbar_<e,t>: (Exists x_e: (ForeignCountry_<e,t>(x_e) & ~(Exists x1_e: ((Cat_<e,t>(x1_e) & From_<(e,e),t>(x1_e, x_e)) & xbar_<e,t>(x1_e))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '⟦[no [[fromP [a fcountry]] cat2]]⟧ = (λ abar_<t,t>: (λ xbar_<e,t>: (Exists x_e: (ForeignCountry_<e,t>(x_e) & abar_<t,t>(~(Exists x1_e: ((Cat_<e,t>(x1_e) & From_<(e,e),t>(x1_e, x_e)) & xbar_<e,t>(x1_e))))))))'/<<t,t>,<<e,t>,t>> and '⟦dance2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[dance2 [no [[fromP [a fcountry]] cat2]]]⟧ = Type mismatch: '⟦dance2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e))))'/<<<e,t>,t>,t> and '⟦[no [[fromP [a fcountry]] cat2]]⟧ = (λ abar_<t,t>: (λ xbar_<e,t>: (Exists x_e: (ForeignCountry_<e,t>(x_e) & abar_<t,t>(~(Exists x1_e: ((Cat_<e,t>(x1_e) & From_<(e,e),t>(x1_e, x_e)) & xbar_<e,t>(x1_e))))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function Application), ⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (λ xbar_<e,t>: (Exists x_e: (ForeignCountry_<e,t>(x_e) & abar_<t,t>(~(Exists x1_e: ((Cat_<e,t>(x1_e) & From_<(e,e),t>(x1_e, x_e)) & xbar_<e,t>(x1_e))))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[dance2 [no [[fromP [a fcountry]] cat2]]]⟧ = Type mismatch: '((λ f_<<<e,t>,t>,t>: (λ arg_<<e,I5726>,t>: (λ abar_<I5724,I5726>: f_<<<e,t>,t>,t>((λ b_<e,t>: arg_<<e,I5726>,t>((λ c_e: abar_<t,I5726>(b_<e,t>(c_e))))))))))((λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e)))))'/<<<e,I5726>,t>,<<I5724,I5726>,t>> and '(λ abar_<t,t>: (λ xbar_<e,t>: (Exists x_e: (ForeignCountry_<e,t>(x_e) & abar_<t,t>(~(Exists x1_e: ((Cat_<e,t>(x1_e) & From_<(e,e),t>(x1_e, x_e)) & xbar_<e,t>(x1_e))))))))'/<<t,t>,<<e,t>,t>> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<t,t>: (λ xbar_<e,t>: (Exists x_e: (ForeignCountry_<e,t>(x_e) & abar_<t,t>(~(Exists x1_e: ((Cat_<e,t>(x1_e) & From_<(e,e),t>(x1_e, x_e)) & xba

Failure!  Only one scoping is generated.  What's going on?

It turns out that the hack/presentational strategy used in the body of Barker's paper to combine NPs with Ds via regular function application doesn't allow for inverse scoping.  This is unsurprising when you think about it as this operation's effect on scope is a bit like what happens with `Disrupt`: continuations are effectively trapped.

We'll have to move to the version in Barker's appendix.  Before doing that, you may want to inspect what composition paths are being found, by looking at `r`.

In [49]:
r[0].tree()

In the appendix, Barker treats quantificational determiners as quantifiers over choice functions of type `<<e,t>,e>`, that is, functions that map properties into individuals.  It turns out, somewhat magically at first glance, that in consequence quantificational determiners are the right type to compose with property-denoting sisters via `CA` or `CA2` and generate a GQ type.  This requires staring at the derivation for a while but the basic idea is that the continuation of a GQ should be type `<e,t>`, and so its input needs to be type `e`, and a choice function for property types bridges this need with the continuized-property-denoting sister.

Strictly speaking, one should do this for `everyone` and `someone` as well, but it is pedagogically simpler not to, so I'll leave them as is.

In [50]:
%%lamb
||a|| = L dbar_<<<e,t>,e>,t> : Exists f_<<e,t>,e> : dbar(f)
||no|| = L dbar_<<<e,t>,e>,t> : ~(Exists f_<<e,t>,e> : dbar(f))
||every|| = L dbar_<<<e,t>,e>,t> : (Forall f_<<e,t>,e> : dbar(f))

$[\![\mathbf{\text{a}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} dbar_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{}} \: . \: \exists{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \: . \: {dbar}({f})$<br />
$[\![\mathbf{\text{no}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} dbar_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{}} \: . \: \neg{} \exists{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \: . \: {dbar}({f})$<br />
$[\![\mathbf{\text{every}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{},t\right\rangle{}} \:=\: $$\lambda{} dbar_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{}} \: . \: \forall{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \: . \: {dbar}({f})$

In [51]:
every * cat2

CompositionResult(results=[⟦[every cat2]⟧ = (λ abar_<e,t>: (Forall f_<<e,t>,e>: abar_<e,t>(f_<<e,t>,e>((λ x_e: Cat_<e,t>(x_e))))))], failures=[⟦[every cat2]⟧ = Type mismatch: '⟦every⟧ = (λ dbar_<<<e,t>,e>,t>: (Forall f_<<e,t>,e>: dbar_<<<e,t>,e>,t>(f_<<e,t>,e>)))'/<<<<e,t>,e>,t>,t> and '⟦cat2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Cat_<e,t>(x_e))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[cat2 every]⟧ = Type mismatch: '⟦cat2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Cat_<e,t>(x_e))))'/<<<e,t>,t>,t> and '⟦every⟧ = (λ dbar_<<<e,t>,e>,t>: (Forall f_<<e,t>,e>: dbar_<<<e,t>,e>,t>(f_<<e,t>,e>)))'/<<<<e,t>,e>,t>,t> conflict (mode: Function Application), ⟦[cat2 every]⟧ = Type mismatch: '((λ f_<<<e,t>,t>,t>: (λ arg_<<e,I5842>,t>: (λ abar_<I5840,I5842>: f_<<<e,t>,t>,t>((λ b_<e,t>: arg_<<e,I5842>,t>((λ c_e: abar_<t,I5842>(b_<e,t>(c_e))))))))))((λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Cat_<e,t>(x_e)))))'/<<<e,I5842>,t>,<<I5840,I5842>,t>> and '(λ dbar_<<<e,t>,e>,t>: (Forall f_<<e,t>,e>: dbar_<<<e,t>,e>,t>(f_<<e,t>,e>)))'/<<<<e,t>,e>,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[cat2 every]⟧ = Type mismatch: '((λ f_<<<e,t>,t>,t>: (λ arg_<<e,t>,I5915>: (λ abar_<I5918,t>: arg_<<e,t>,I5915>((λ c_e: f_<<<e,t>,t>,t>((λ b_<e,t>: abar_<t,t>(b_<e,t>(c_e))))))))))((λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Cat_<e,t>(x_e)))))'/<<<e,t>,I5915>,<<I5918,t>,I5915>> and '(λ dbar_<<<e,t>,e>,t>: (Forall f_<<e,t>,e>: dbar_<<<e,t>,e>,t>(f_<<e,t>,e>)))'/<<<<e,t>,e>,t>,t> conflict (mode: Function argument combination (unification failed))])

In [52]:
(every * cat2)[0].tree(derivations=True)

,"$[\![\mathbf{\text{cat2}}]\!]^{}_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}}$"
$=$,"${[\lambda{} x_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}({x})]}(\lambda{} x_{e} \: . \: {Cat}({x}))$"
$=$,"$\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {Cat}({x}))$"
,"$[\![\mathbf{\text{[every cat2]}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}}$"
$=$,"${[\lambda{} f_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} arg_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} abar_{\left\langle{}e,t\right\rangle{}} \: . \: {f}(\lambda{} b_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \: . \: {arg}(\lambda{} c_{\left\langle{}e,t\right\rangle{}} \: . \: {abar}({b}({c}))))]}(\lambda{} dbar_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{}} \: . \: \forall{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \: . \: {dbar}({f}))(\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {Cat}({x})))$"
$=$,"${[\lambda{} f_{\left\langle{}\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} arg_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} abar_{\left\langle{}e,t\right\rangle{}} \: . \: {f}(\lambda{} b_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \: . \: {arg}(\lambda{} c_{\left\langle{}e,t\right\rangle{}} \: . \: {abar}({b}({c}))))]}(\lambda{} dbar_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{},t\right\rangle{}} \: . \: \forall{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \: . \: {dbar}({f}))(\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {Cat}({x})))$"
$=$,"${[\lambda{} arg_{\left\langle{}\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{},t\right\rangle{}} \: . \: \lambda{} abar_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \: . \: {arg}(\lambda{} c_{\left\langle{}e,t\right\rangle{}} \: . \: {abar}({f}({c})))]}(\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {Cat}({x})))$"
$=$,"$\lambda{} abar_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \: . \: {[\lambda{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},t\right\rangle{}} \: . \: {f}(\lambda{} x_{e} \: . \: {Cat}({x}))]}(\lambda{} c_{\left\langle{}e,t\right\rangle{}} \: . \: {abar}({f}({c})))$"
$=$,"$\lambda{} abar_{\left\langle{}e,t\right\rangle{}} \: . \: \forall{} f_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \: . \: {abar}({f}(\lambda{} x_{e} \: . \: {Cat}({x})))$"


It is worth pausing for a moment to contemplate the way this works in the above derivation.

With this framework we can return to our set of inverse linking examples.  Note that some of these can be extremely slow to render and we are getting an exponential explosion because CA and CA2 can both apply at any stage; this can be filtered if necessary using the `eliminate_dups` function on `CompositionResult`s, but it is instructive to see all composition paths so it is not done by default.

In [53]:
tfilter((no * (cat2 * (fromP * france))) * dance2)

CompositionResult(results=[⟦[dance2 [no [[fromP france] cat2]]]⟧ = (λ abar_<t,t>: ~(Exists f_<<e,t>,e>: abar_<t,t>(Dance_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, France_e))))))))], failures=[⟦[[no [[fromP france] cat2]] dance2]⟧ = Type mismatch: '⟦[no [[fromP france] cat2]]⟧ = (λ abar_<e,t>: ~(Exists f_<<e,t>,e>: abar_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, France_e)))))))'/<<e,t>,t> and '⟦dance2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[[no [[fromP france] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<e,t>: ~(Exists f_<<e,t>,e>: abar_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, France_e)))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP france] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<e,t>: ~(Exists f_<<e,t>,e>: abar_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, France_e)))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[dance2 [no [[fromP france] cat2]]]⟧ = ~(Exists f_<<e,t>,e>: Dance_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, France_e))))))])

In [54]:
r = tfilter((no * (cat2 * (fromP * (a * fcountry)))) * dance2)
r

CompositionResult(results=[⟦[dance2 [no [[fromP [a fcountry]] cat2]]]⟧ = (λ abar_<t,t>: ~(Exists f_<<e,t>,e>: (Exists f1_<<e,t>,e>: abar_<t,t>(Dance_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f1_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))), ⟦[dance2 [no [[fromP [a fcountry]] cat2]]]⟧ = (λ abar_<t,t>: (Exists f_<<e,t>,e>: ~(Exists f1_<<e,t>,e>: abar_<t,t>(Dance_<e,t>(f1_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e))))))))))))], failures=[⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '⟦[no [[fromP [a fcountry]] cat2]]⟧ = (λ abar_<e,t>: ~(Exists f_<<e,t>,e>: (Exists f1_<<e,t>,e>: abar_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f1_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> and '⟦dance2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<e,t>: ~(Exists f_<<e,t>,e>: (Exists f1_<<e,t>,e>: abar_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f1_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<e,t>: ~(Exists f_<<e,t>,e>: (Exists f1_<<e,t>,e>: abar_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f1_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '⟦[no [[fromP [a fcountry]] cat2]]⟧ = (λ abar_<e,t>: (Exists f_<<e,t>,e>: ~(Exists f1_<<e,t>,e>: abar_<e,t>(f1_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> and '⟦dance2⟧ = (λ f_<<e,t>,t>: f_<<e,t>,t>((λ x_e: Dance_<e,t>(x_e))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<e,t>: (Exists f_<<e,t>,e>: ~(Exists f1_<<e,t>,e>: abar_<e,t>(f1_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP [a fcountry]] cat2]] dance2]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<e,t>: (Exists f_<<e,t>,e>: ~(Exists f1_<<e,t>,e>: abar_<e,t>(f1_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[dance2 [no [[fromP [a fcountry]] cat2]]]⟧ = ~(Exists f_<<e,t>,e>: (Exists f1_<<e,t>,e>: Dance_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f1_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))), ⟦[dance2 [no [[fromP [a fcountry]] cat2]]]⟧ = (Exists f_<<e,t>,e>: ~(Exists f1_<<e,t>,e>: Dance_<e,t>(f1_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e))))))))))])

Three quantifiers:

In [55]:
r[0].tree()

In [56]:
r2 = tfilter((no * (cat2 * (fromP * (a * fcountry)))) * (saw * everyone))
r2 

CompositionResult(results=[⟦[[saw everyone] [no [[fromP [a fcountry]] cat2]]]⟧ = (λ abar_<t,t>: (Forall x_e: ~(Exists f_<<e,t>,e>: (Exists f1_<<e,t>,e>: abar_<t,t>(Saw_<(e,e),t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f1_<<e,t>,e>((λ x1_e: ForeignCountry_<e,t>(x1_e))))))), x_e)))))), ⟦[[saw everyone] [no [[fromP [a fcountry]] cat2]]]⟧ = (λ abar_<t,t>: ~(Exists f_<<e,t>,e>: (Exists f1_<<e,t>,e>: (Forall x_e: abar_<t,t>(Saw_<(e,e),t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f1_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e))))))), x_e)))))), ⟦[[saw everyone] [no [[fromP [a fcountry]] cat2]]]⟧ = (λ abar_<t,t>: (Forall x_e: (Exists f_<<e,t>,e>: ~(Exists f1_<<e,t>,e>: abar_<t,t>(Saw_<(e,e),t>(f1_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f_<<e,t>,e>((λ x1_e: ForeignCountry_<e,t>(x1_e))))))), x_e)))))), ⟦[[saw everyone] [no [[fromP [a fcountry]] cat2]]]⟧ = (λ abar_<t,t>: (Exists f_<<e,t>,e>: ~(Exists f1_<<e,t>,e>: (Forall x_e: abar_<t,t>(Saw_<(e,e),t>(f1_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e))))))), x_e))))))], failures=[⟦[[no [[fromP [a fcountry]] cat2]] [saw everyone]]⟧ = Type mismatch: '⟦[no [[fromP [a fcountry]] cat2]]⟧ = (λ abar_<e,t>: ~(Exists f_<<e,t>,e>: (Exists f1_<<e,t>,e>: abar_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f1_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> and '⟦[saw everyone]⟧ = (λ abar_<<e,t>,t>: (Forall x_e: abar_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e)))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[[no [[fromP [a fcountry]] cat2]] [saw everyone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<e,t>: ~(Exists f_<<e,t>,e>: (Exists f1_<<e,t>,e>: abar_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f1_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP [a fcountry]] cat2]] [saw everyone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',X'''>,X5>>> and '(λ abar_<e,t>: ~(Exists f_<<e,t>,e>: (Exists f1_<<e,t>,e>: abar_<e,t>(f_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f1_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP [a fcountry]] cat2]] [saw everyone]]⟧ = Type mismatch: '⟦[no [[fromP [a fcountry]] cat2]]⟧ = (λ abar_<e,t>: (Exists f_<<e,t>,e>: ~(Exists f1_<<e,t>,e>: abar_<e,t>(f1_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> and '⟦[saw everyone]⟧ = (λ abar_<<e,t>,t>: (Forall x_e: abar_<<e,t>,t>((λ x1_e: Saw_<(e,e),t>(x1_e, x_e)))))'/<<<e,t>,t>,t> conflict (mode: Function Application), ⟦[[no [[fromP [a fcountry]] cat2]] [saw everyone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X4>,X5>: (λ arg_<<X,X'''>,X4>: (λ abar_<X'',X'''>: f_<<<X,X'>,X4>,X5>((λ b_<X,X'>: arg_<<X,X'''>,X4>((λ c_X: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X4>,X5>,<<<X,X'''>,X4>,<<X'',X'''>,X5>>> and '(λ abar_<e,t>: (Exists f_<<e,t>,e>: ~(Exists f1_<<e,t>,e>: abar_<e,t>(f1_<<e,t>,e>((λ y_e: (Cat_<e,t>(y_e) & From_<(e,e),t>(y_e, f_<<e,t>,e>((λ x_e: ForeignCountry_<e,t>(x_e)))))))))))'/<<e,t>,t> conflict (mode: Function argument combination (unification failed)), ⟦[[no [[fromP [a fcountry]] cat2]] [saw everyone]]⟧ = Type mismatch: '(λ f_<<<X,X'>,X'''>,X4>: (λ arg_<<X,X4>,X5>: (λ abar_<X'',X'''>: arg_<<X,X4>,X5>((λ c_X: f_<<<X,X'>,X'''>,X4>((λ b_<X,X'>: abar_<X'',X'''>(b_<X,X'>(c_X)))))))))'/<<<<X,X'>,X'''>,X4>,<<<X,X4>,X5>,<<X'',

We have:
 * 0: everyone >> no cat >> a foreign country
 * 1: no cat >> a foreign country >> everyone
 * 2: everyone >> a foreign country >> no cat
 * 3: a foreign country >> no cat >> everyone
 
What is missing?
 * no cat >> everyone >> a foreign country
 * a foreign country >> everyone >> no cat
 
These are exactly the cases where something would split the scope of the two quantifiers in the subject DP, and empirically, these readings are supposed to be absent in general (this observation is due to May).  This demonstrates what Barker calls the _Integrity Constraint_, which is that when scopal elements form a constituent together, they can't scope independently w.r.t. scopal elements outside that constituent.  Intuitively, their scope is compositionally determined inside the smallest constituent they are a member of, and can't be changed or interrupted after that.

Arguably, the derivation of this constraint from extremely general principles is the most important feature of this analysis.

This concludes this fragment; I have left out discussion of conjunction, and of the mathematical parts of Barker's paper.